In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Define a list of stock symbols
stock_symbols = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOGL", "BRK-B", "JNJ", "UNH", "NVDA", "META", "PG", "JPM", "XOM", "V", "HD", "CVX", "MA", "ABBV", "PFE", "BAC"]

# Initialize models
models = {
    'SVR': SVR(),
    'LightGBM': lgb.LGBMRegressor(),
    'GBR': GradientBoostingRegressor(),
    'RandomForest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
}

# Loop over each stock symbol
for symbol in stock_symbols:
    # Load closing prices and volume data using Yahoo Finance
    stock_data = yf.download(symbol, start="2020-01-01", end="2021-12-31")

    # Calculate log returns
    stock_data['Log_Return'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))

    # Create a new column 'Forward_Return' with the 10-day forward log returns
    stock_data['Forward_Return'] = stock_data['Log_Return'].shift(-10)

    # Calculate 10-day moving average
    stock_data['MA_10'] = stock_data['Close'].rolling(window=10).mean()

    # Calculate 10-day exponential moving average (EMA)
    stock_data['EMA_10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()

    # Calculate 10-day Relative Strength Index (RSI)
    delta = stock_data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=10).mean()
    avg_loss = loss.rolling(window=10).mean()
    rs = avg_gain / avg_loss
    stock_data['RSI_10'] = 100 - (100 / (1 + rs))

    # Load S&P 500 prices and 10-year Treasury yield data using Yahoo Finance
    sp500_data = yf.download('^GSPC', start="2020-01-01", end="2021-12-31")
    treasury_yield_data = yf.download('^TNX', start="2020-01-01", end="2021-12-31")

    # Merge S&P 500 prices and Treasury yield data with the stock data
    stock_data = stock_data.merge(sp500_data['Close'], left_index=True, right_index=True, suffixes=('', '_SP500'))
    stock_data = stock_data.merge(treasury_yield_data['Close'], left_index=True, right_index=True, suffixes=('', '_Yield'))

    # Calculate 10-day rolling moving average of returns
    stock_data['Rolling_MA_10_Return'] = stock_data['Log_Return'].rolling(window=10).mean()

    # Drop rows with NaN values (due to the rolling, EMA calculations, and shift operation)
    stock_data.dropna(inplace=True)

    # Define the feature columns
    features = ['Close', 'Log_Return', 'Volume', 'MA_10', 'EMA_10', 'RSI_10', 'Close_SP500', 'Close_Yield', 'Rolling_MA_10_Return']

    # Calculate the index to split the data
    split_index = int(len(stock_data) * 0.8)

    # Split the data into training and testing sets
    train_data = stock_data.iloc[:split_index]
    test_data = stock_data.iloc[split_index:]

    # Separate the features and target for training and testing
    X_train = train_data[features]
    y_train = train_data['Forward_Return']
    X_test = test_data[features]
    y_test = test_data['Forward_Return']

    # Normalize the data for LSTM
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=5)  # Adjust the number of components as needed
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_pca.shape[1], 1)))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')

    # Reshape the data for LSTM
    X_train_lstm = X_train_pca.reshape(X_train_pca.shape[0], X_train_pca.shape[1], 1)
    X_test_lstm = X_test_pca.reshape(X_test_pca.shape[0], X_test_pca.shape[1], 1)

    # Train the LSTM model
    lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

    # Predict using the LSTM model
    lstm_predictions = lstm_model.predict(X_test_lstm)

    # Initialize models and store predictions
    model_predictions = {}
    for model_name, model in models.items():
        # Apply PCA to reduce dimensionality for other models
        pca = PCA(n_components=5)  # Adjust the number of components as needed
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Train the model on the entire training set
        model.fit(X_train_pca, y_train)

        # Apply PCA to the test set
        model_predictions[model_name] = model.predict(X_test_pca)

    # Add LSTM predictions to the ensemble predictions
    model_predictions['LSTM'] = lstm_predictions.flatten()

    # Create an ensemble model by fitting a linear regression to individual model predictions
    ensemble_model = LinearRegression()
    ensemble_model.fit(np.array(list(model_predictions.values())).T, y_test)

    # Use the ensemble model to make predictions
    ensemble_predictions = ensemble_model.predict(np.array(list(model_predictions.values())).T)

    # Plotting for the current stock
    plt.figure(figsize=(12, 8))
    plt.plot(y_test.index, y_test.values, label='Actual', linestyle='-', marker='o', alpha=0.5)
    plt.plot(y_test.index, ensemble_predictions, label='Ensemble (Linear Regression)', linestyle='-', marker='.', color='black')
    plt.plot(y_test.index, lstm_predictions, label='Predicted (LSTM)', linestyle='-', marker='x', color='red')
    for model_name, predictions in model_predictions.items():
        if model_name != 'LSTM':
            plt.plot(y_test.index, predictions, label=f'Predicted ({model_name})', linestyle='-', marker='x')
    plt.plot(y_test.index, test_data['Rolling_MA_10_Return'], label='Rolling_MA_10_Return', linestyle='-', marker='^', color='green')
    plt.xlabel("Date")
    plt.ylabel("10-Day Forward Log Returns")
    plt.title(f"Actual vs. Predicted Values for {symbol}")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Print evaluation results for the ensemble model and individual models
    print(f"Results for {symbol}:")
    ensemble_mae = mean_absolute_error(y_test, ensemble_predictions)
    ensemble_rmse = mean_squared_error(y_test, ensemble_predictions, squared=False)
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    print(f'Ensemble (Linear Regression) - MAE: {ensemble_mae}, RMSE: {ensemble_rmse}, R2: {ensemble_r2}')
    for model_name, predictions in model_predictions.items():
        if model_name != 'LSTM':
            mae = mean_absolute_error(y_test, predictions)
            rmse = mean_squared_error(y_test, predictions, squared=False)
            r2 = r2_score(y_test, predictions)
            print(f'{model_name} - MAE: {mae}, RMSE: {rmse}, R2: {r2}')


Output hidden; open in https://colab.research.google.com to view.

In [2]:
#1-day ahead forecasts
import pandas as pd
import numpy as np
import yfinance as yf
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Define a list of stock symbols
stock_symbols = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOGL", "BRK-B", "JNJ", "UNH", "NVDA", "META", "PG", "JPM", "XOM", "V", "HD", "CVX", "MA", "ABBV", "PFE", "BAC"]

# Initialize models
models = {
    'SVR': SVR(),
    'LightGBM': lgb.LGBMRegressor(),
    'GBR': GradientBoostingRegressor(),
    'RandomForest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
}

# Loop over each stock symbol
for symbol in stock_symbols:
    # Load closing prices and volume data using Yahoo Finance
    stock_data = yf.download(symbol, start="2020-01-01", end="2021-12-31")

    # Calculate log returns
    stock_data['Log_Return'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))

    # Create a new column 'Forward_Return' with the 1-day forward log returns
    stock_data['Forward_Return'] = stock_data['Log_Return'].shift(-1)

    # Calculate 10-day moving average
    stock_data['MA_10'] = stock_data['Close'].rolling(window=10).mean()

    # Calculate 10-day exponential moving average (EMA)
    stock_data['EMA_10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()

    # Calculate 10-day Relative Strength Index (RSI)
    delta = stock_data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=10).mean()
    avg_loss = loss.rolling(window=10).mean()
    rs = avg_gain / avg_loss
    stock_data['RSI_10'] = 100 - (100 / (1 + rs))

    # Load S&P 500 prices and 10-year Treasury yield data using Yahoo Finance
    sp500_data = yf.download('^GSPC', start="2020-01-01", end="2021-12-31")
    treasury_yield_data = yf.download('^TNX', start="2020-01-01", end="2021-12-31")

    # Merge S&P 500 prices and Treasury yield data with the stock data
    stock_data = stock_data.merge(sp500_data['Close'], left_index=True, right_index=True, suffixes=('', '_SP500'))
    stock_data = stock_data.merge(treasury_yield_data['Close'], left_index=True, right_index=True, suffixes=('', '_Yield'))

    # Calculate 10-day rolling moving average of returns
    stock_data['Rolling_MA_10_Return'] = stock_data['Log_Return'].rolling(window=10).mean()

    # Drop rows with NaN values (due to the rolling, EMA calculations, and shift operation)
    stock_data.dropna(inplace=True)

    # Define the feature columns
    features = ['Close', 'Log_Return', 'Volume', 'MA_10', 'EMA_10', 'RSI_10', 'Close_SP500', 'Close_Yield', 'Rolling_MA_10_Return']

    # Calculate the index to split the data
    split_index = int(len(stock_data) * 0.8)

    # Split the data into training and testing sets
    train_data = stock_data.iloc[:split_index]
    test_data = stock_data.iloc[split_index:]

    # Separate the features and target for training and testing
    X_train = train_data[features]
    y_train = train_data['Forward_Return']
    X_test = test_data[features]
    y_test = test_data['Forward_Return']

    # Normalize the data for LSTM
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=5)  # Adjust the number of components as needed
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_pca.shape[1], 1)))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')

    # Reshape the data for LSTM
    X_train_lstm = X_train_pca.reshape(X_train_pca.shape[0], X_train_pca.shape[1], 1)
    X_test_lstm = X_test_pca.reshape(X_test_pca.shape[0], X_test_pca.shape[1], 1)

    # Train the LSTM model
    lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

    # Predict using the LSTM model
    lstm_predictions = lstm_model.predict(X_test_lstm)

    # Initialize models and store predictions
    model_predictions = {}
    for model_name, model in models.items():
        # Apply PCA to reduce dimensionality for other models
        pca = PCA(n_components=5)  # Adjust the number of components as needed
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Train the model on the entire training set
        model.fit(X_train_pca, y_train)

        # Apply PCA to the test set
        model_predictions[model_name] = model.predict(X_test_pca)

    # Add LSTM predictions to the ensemble predictions
    model_predictions['LSTM'] = lstm_predictions.flatten()

    # Create an ensemble model by fitting a linear regression to individual model predictions
    ensemble_model = LinearRegression()
    ensemble_model.fit(np.array(list(model_predictions.values())).T, y_test)

    # Use the ensemble model to make predictions
    ensemble_predictions = ensemble_model.predict(np.array(list(model_predictions.values())).T)

    # Plotting for the current stock
    plt.figure(figsize=(12, 8))
    plt.plot(y_test.index, y_test.values, label='Actual', linestyle='-', marker='o', alpha=0.5)
    plt.plot(y_test.index, ensemble_predictions, label='Ensemble (Linear Regression)', linestyle='-', marker='.', color='black')
    plt.plot(y_test.index, lstm_predictions, label='Predicted (LSTM)', linestyle='-', marker='x', color='red')
    for model_name, predictions in model_predictions.items():
        if model_name != 'LSTM':
            plt.plot(y_test.index, predictions, label=f'Predicted ({model_name})', linestyle='-', marker='x')
    plt.plot(y_test.index, test_data['Rolling_MA_10_Return'], label='Rolling_MA_10_Return', linestyle='-', marker='^', color='green')
    plt.xlabel("Date")
    plt.ylabel("1-Day Forward Log Returns")
    plt.title(f"Actual vs. Predicted Values for {symbol}")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Print evaluation results for the ensemble model and individual models
    print(f"Results for {symbol}:")
    ensemble_mae = mean_absolute_error(y_test, ensemble_predictions)
    ensemble_rmse = mean_squared_error(y_test, ensemble_predictions, squared=False)
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    print(f'Ensemble (Linear Regression) - MAE: {ensemble_mae}, RMSE: {ensemble_rmse}, R2: {ensemble_r2}')
    for model_name, predictions in model_predictions.items():
        if model_name != 'LSTM':
            mae = mean_absolute_error(y_test, predictions)
            rmse = mean_squared_error(y_test, predictions, squared=False)
            r2 = r2_score(y_test, predictions)
            print(f'{model_name} - MAE: {mae}, RMSE: {rmse}, R2: {r2}')


Output hidden; open in https://colab.research.google.com to view.